In [1]:
import numpy as np
import pandas as pd

In [2]:
import skimage as sk


In [3]:
import sys
sys.path.append(r'C:\Users\Kevin\repos\ReipurthBallyProject')

In [4]:
from src.r_d_src.coo_utils import coo2df

In [5]:
satsuki_df = coo2df(r'C:\Users\Kevin\repos\ReipurthBallyProject\SubaruCoordinateMaps\satsuki.coo')

In [6]:
satsuki_df

,x_ref,y_ref,x_in,y_in
0,1986.0542,94.003948,2004.39440,68.327594
1,1945.7056,53.655391,1960.37780,27.979037
2,1718.2865,119.680300,1732.95870,94.003948
3,1718.2865,71.995644,1729.29060,49.987341
4,1751.2989,71.995644,1765.97110,46.319290
...,...,...,...,...
75,938.1168,3801.921400,947.48534,3810.438300
76,1080.3482,3908.382000,1088.01340,3915.195500
77,1575.7865,3991.431400,1584.30330,3999.005900
78,1880.6579,4051.407400,1890.02250,4058.307600


In [7]:
src = np.array([satsuki_df.x_in, satsuki_df.y_in]).T
dst = np.array([satsuki_df.x_ref, satsuki_df.y_ref]).T

In [8]:
tran = sk.transform.estimate_transform('polynomial', src,dst, 3)

In [9]:
def rmse(resid):
    RMSE = np.sqrt((resid**2).mean())
    return RMSE

In [10]:
resid = tran.residuals(src, dst)
rmse(resid)

1.2059969731883307

In [11]:
import os
coo_maps = r'C:\Users\Kevin\repos\ReipurthBallyProject\SubaruCoordinateMaps'
for detector in os.listdir(coo_maps):
    coo_path = os.path.join(coo_maps, detector)
    coo_df = coo2df(coo_path)
    src = np.array([coo_df.x_in, coo_df.y_in]).T
    dst = np.array([coo_df.x_ref, coo_df.y_ref]).T
    tran = sk.transform.estimate_transform('polynomial', src,dst, 3)
    resid = tran.residuals(src, dst)
    RMSE = rmse(resid)
    print(f'Detector: {detector}, Number of Points: {len(resid)}, RMSE: {RMSE}')


Detector: chihiro.coo, Number of Points: 68, RMSE: 2.0524217551510096
Detector: clarisse.coo, Number of Points: 78, RMSE: 3.3100906165179977
Detector: fio.coo, Number of Points: 54, RMSE: 3.8304669770356994
Detector: kiki.coo, Number of Points: 51, RMSE: 2.0682778001658164
Detector: nausicaa.coo, Number of Points: 51, RMSE: 1.1590772147647659
Detector: ponyo.coo, Number of Points: 70, RMSE: 1.6700033305330588
Detector: san.coo, Number of Points: 92, RMSE: 2.5453589806923773
Detector: satsuki.coo, Number of Points: 80, RMSE: 1.2059969731883307
Detector: sheeta.coo, Number of Points: 53, RMSE: 3.7128780551456586
Detector: sophie.coo, Number of Points: 49, RMSE: 10.599157412578823
Detector: sophie.coo.bak, Number of Points: 50, RMSE: 12.663184747056043


In [12]:
for detector in ['satsuki.coo']:
    coo_path = os.path.join(coo_maps, detector)
    coo_df = coo2df(coo_path)
    src = np.array([coo_df.x_in, coo_df.y_in]).T
    dst = np.array([coo_df.x_ref, coo_df.y_ref]).T
    tran = sk.transform.estimate_transform('polynomial', src,dst, 3)
    resid = tran.residuals(src, dst)
    RMSE = rmse(resid)
    print(f'Detector: {detector}, Number of  RMSE: {RMSE}')

Detector: satsuki.coo, Number of  RMSE: 1.2059969731883307


In [13]:
resid.max(), resid.argmax()

(3.9639358902060486, 3)

In [14]:
np.allclose(np.sqrt(((dst - tran(src))**2).sum(axis=1)), resid)

True

In [15]:
def df2reg(df, regpath, x='x_in', y='y_in', radius=8, nameroot='Region'):
    reghdr =[ '# Region file format: DS9 version 4.1',
             'global color=green dashlist=8 3 width=1 font="helvetica 10 normal roman" select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1',
            'physical']
    x_col = 1 if x == 'x_ref' else 3
    y_col = 2 if y == 'y_ref' else 4
    with open(regpath, 'w') as reg:
        for hdr in reghdr:
            reg.write(hdr+'\n')
        for i, r in enumerate(df.itertuples()):
            title = '{' + f'{nameroot}_{i:03d}' + '}'
            circ = f'Circle({r[x_col]}, {r[y_col]}, {radius}) # text={title}'
            reg.write(circ+'\n')

        



In [16]:
detector = 'nausicaa'
coo_path = os.path.join(coo_maps, detector+'.coo')
coo_df = coo2df(coo_path)

regpath = os.path.join(r'C:\Users\Kevin\repos\ReipurthBallyProject\data\M8\N-A-L656',detector+'.reg')

df2reg(coo_df, regpath)

In [17]:
import sep
from astropy.io import fits

In [18]:
def find_objects(unbiased_image, thresh = 3):
    with fits.open(unbiased_image) as f:
        hdr = f[0].header.copy()
        img = f[0].data.copy()
    img = img.byteswap().newbyteorder()
    bkg = sep.Background(img)
    bkg_img = bkg.back() #2d array of background

    img_noback = img - bkg
    objects = sep.extract(img_noback, thresh=thresh, err = bkg.globalrms)
    objects_df = pd.DataFrame(objects)
    return objects_df

In [19]:
img_path = r'C:\Users\Kevin\repos\ReipurthBallyProject\data\M8\N-A-L656\no_bias\SUPA01564800.fits'
object_df = find_objects(img_path,thresh=50)


In [20]:
object_df

,thresh,npix,tnpix,xmin,xmax,ymin,ymax,x,y,x2,...,cxy,cflux,flux,cpeak,peak,xcpeak,ycpeak,xpeak,ypeak,flag
0,6641.522461,1871,1800,8,1044,0,2,513.168784,0.409608,32609.195080,...,0.001279,3.232333e+07,3.828258e+07,25922.291016,40604.335938,486,0,486,0,3
1,6641.522461,65,60,784,804,0,4,794.071595,1.303218,10.433083,...,0.073499,2.317405e+06,2.478738e+06,120508.148438,159052.906250,794,2,794,1,3
2,6641.522461,1613,1351,693,1389,0,2,975.752265,0.421780,29137.773239,...,-0.001855,2.651141e+07,3.153486e+07,32743.623047,56744.062500,1087,0,1087,0,3
3,6641.522461,468,359,0,187,0,4,63.028453,0.451371,2396.897449,...,-0.002884,6.855893e+06,8.161418e+06,28992.990234,50182.976562,1,0,0,0,3
4,6641.522461,1562,1399,1275,2047,0,3,1819.077559,0.452636,80158.781442,...,0.003819,1.736041e+07,2.140818e+07,24888.074219,38575.242188,1431,0,1431,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282,6641.522461,36,35,1408,1413,4104,4110,1410.696709,4106.787604,1.866789,...,-0.061597,5.972308e+05,6.277271e+05,33402.707031,40053.761719,1411,4107,1411,4107,0
283,6641.522461,37,32,959,964,4126,4132,961.429722,4128.984477,1.870247,...,-0.037491,5.438982e+05,5.708504e+05,29573.142578,34777.199219,961,4129,961,4129,0
284,6641.522461,13,13,1941,1944,4164,4167,1942.622248,4165.332839,0.903700,...,0.295505,1.139122e+05,1.274280e+05,11424.018555,13601.125977,1943,4165,1943,4165,0
285,6641.522461,12,10,1312,1316,4174,4176,1314.139373,4175.360553,1.444894,...,0.178911,1.361231e+05,1.733078e+05,16915.134766,29253.648438,1314,4176,1314,4176,2


In [21]:
reg_df = object_df[['x','y']].rename(columns={'x':'x_ref', 'y':'y_ref'})
#convert to fits (1-relative) indexing
reg_df.x_ref += 1
reg_df.y_ref += 1
df2reg(reg_df, r'C:\Users\Kevin\repos\ReipurthBallyProject\data\M8\N-A-L656\region\SUPA01564800.reg', x='x_ref', y='y_ref')

In [22]:
def coo2reg(coo, reg_path, nameroot='Star'):
    reghdr =[ '# Region file format: DS9 version 4.1',
            'global color=green dashlist=8 3 width=1 font="helvetica 10 normal roman" select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1',
        'physical']
    
    if type(coo) == str:
        coo_df = coo2df(coo_path)
    elif type(coo) == pd.DataFrame:
        coo_df = coo
    else:
        raise ValueError('Invalid coo type')

    with open(reg_path, 'w') as reg:
        for hdr in reghdr:
            reg.write(hdr+'\n')
        for i,pair in enumerate(coo_df.itertuples()):
            title = '{' + f'{nameroot}_{i:03d}' + '}'
            circ = f'Circle({pair.x_in}, {pair.y_in}, 8) # text={title}'
            reg.write(circ+'\n')
            circ = f'Circle({pair.x_ref}, {pair.y_ref}, 8) # color=blue'
            reg.write(circ+'\n')
            line = f'line( {pair.x_in}, {pair.y_in}, {pair.x_ref}, {pair.y_ref}) # line=0 1'
            reg.write(line+'\n')
    

In [23]:
coo_maps = r'C:\Users\Kevin\repos\ReipurthBallyProject\SubaruCoordinateMaps'
regpath = r'C:\Users\Kevin\repos\ReipurthBallyProject\data\M8\N-A-L656\region\SUPA01564800_coo.reg'
detector = 'nausicaa'
coo2reg(os.path.join(coo_maps, detector+'.coo'), regpath)

In [24]:
type(regpath) == int, type(object_df)==pd.DataFrame

(False, True)

In [25]:
#get transform from coo file
coo_maps = r'C:\Users\Kevin\repos\ReipurthBallyProject\SubaruCoordinateMaps'
detector = 'nausicaa'
coo_path = os.path.join(coo_maps, detector+'.coo')
coo_df = coo2df(coo_path)
src = np.array([coo_df.x_in, coo_df.y_in]).T
dst = np.array([coo_df.x_ref, coo_df.y_ref]).T
tran = sk.transform.estimate_transform('polynomial', src,dst, 3)
#apply transform to stars extracted from image
img_path = r'C:\Users\Kevin\repos\ReipurthBallyProject\data\M8\N-A-L656\no_bias\SUPA01564800.fits'
object_df = find_objects(img_path,thresh=50)
src = np.array([object_df.x+1, object_df.y+1]).T
dst = tran(src)
# write out new region file
coo_df = pd.DataFrame({'x_in':src[:,0], 'y_in':src[:,1], 'x_ref':dst[:,0], 'y_ref':dst[:,1]})
coo2reg(coo_df, r'C:\Users\Kevin\repos\ReipurthBallyProject\data\M8\N-A-L656\region\nausicaa_coo.reg')


In [26]:
def warp_image(img_path, out_path, detector = 'nausicaa',
               coo_maps =r'C:\Users\Kevin\repos\ReipurthBallyProject\SubaruCoordinateMaps' ):
    coo_path = os.path.join(coo_maps, detector+'.coo')

    coo_df = coo2df(coo_path)
    src = np.array([coo_df.x_in, coo_df.y_in]).T
    dst = np.array([coo_df.x_ref, coo_df.y_ref]).T

    # we actually need the inverse transform, to get it, swap dst and src as below
    tran = sk.transform.estimate_transform('polynomial', dst, src, 3)

    with fits.open(img_path) as f:
        img = f[0].data.copy()
        hdr = f[0].header.copy()

    img_new = sk.transform.warp(img, tran)
    phdu = fits.PrimaryHDU(data = img_new, header=hdr)

    phdu.writeto(out_path, overwrite=True)

In [27]:
img_path = r'C:\Users\Kevin\repos\ReipurthBallyProject\data\M8\N-A-L656\no_bias\SUPA01564800.fits'
out_path = r'C:\Users\Kevin\repos\ReipurthBallyProject\data\M8\N-A-L656\region\SUPA01564800.fits'
warp_image(img_path, out_path)

In [28]:
len(object_df)

287

In [29]:
from astropy.table import Table

In [30]:
tbl_path = r"C:\Users\Kevin\repos\ReipurthBallyProject\data\M8\N-A-L656\region\ds9.xml"
ds9tbl = Table.read(tbl_path)

In [31]:
ds9tbl

_RAJ2000,_DEJ2000,RA_ICRS,e_RA_ICRS,DE_ICRS,e_DE_ICRS,Source,Plx,e_Plx,pmRA,e_pmRA,pmDE,e_pmDE,Dup,FG,e_FG,Gmag,e_Gmag,FBP,e_FBP,BPmag,e_BPmag,FRP,e_FRP,RPmag,e_RPmag,BP-RP,RV,e_RV,Teff,AG,E_BP-RP_,Rad,Lum
deg,deg,deg,mas,deg,mas,,mas,mas,mas / yr,mas / yr,mas / yr,mas / yr,,,,mag,mag,,,mag,mag,,,mag,mag,mag,km / s,km / s,K,mag,mag,Rsun,Lsun
float64,float64,float64,float64,float64,float64,int64,float64,float32,float64,float32,float64,float32,uint8,float32,float32,float64,float64,float32,float32,float64,float64,float32,float32,float64,float64,float64,float64,float32,float64,float32,float32,float32,float64
270.6607305152326,-24.3466958686422,270.66070255571,0.1104,-24.34672267461,0.0990,4066019808008492416,0.8285,0.1082,-5.916,0.205,-6.226,0.161,0,2700,3.311,17.1099,0.0013,1274,24.5,17.5886,0.0209,2832,31.88,16.1316,0.0122,1.4570,--,--,--,--,--,--,--
270.6521413634034,-24.3480323559917,270.65214136340,0.7679,-24.34803235599,0.7437,4066019803690596480,--,--,--,--,--,--,0,321.8,2.904,19.4194,0.0098,482,30.11,18.6437,0.0678,966,30.33,17.2994,0.0341,1.3443,--,--,--,--,--,--,--
270.6521034038814,-24.3466744221536,270.65209726287,0.2828,-24.34667888617,0.2823,4066019803681079424,0.7684,0.2977,-1.299,0.480,-1.037,0.401,0,813.1,2.809,18.4130,0.0038,634.5,21.3,18.3453,0.0365,1198,20.58,17.0661,0.0187,1.2792,--,--,--,--,--,--,--
270.6669860809204,-24.3463621783497,270.66698608092,6.6455,-24.34636217835,6.0490,4066019807999188992,--,--,--,--,--,--,0,229.3,3.867,19.7874,0.0183,412.2,39.66,18.8136,0.1045,698.7,34.72,17.6512,0.0540,1.1624,--,--,--,--,--,--,--
270.6638630774177,-24.3433482074884,270.66387017871,0.9269,-24.34333812808,0.8399,4066019803690608128,1.6992,0.9087,1.503,1.982,2.341,1.724,0,201.1,1.8,19.9300,0.0097,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--
270.6846830664570,-24.3260169878842,270.68468306646,5.7222,-24.32601698788,2.5360,4066020220316058368,--,--,--,--,--,--,0,131.6,2.248,20.3899,0.0185,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--
270.6926632528732,-24.3279138863670,270.69266325287,2.5770,-24.32791388637,1.9835,4066020151597899648,--,--,--,--,--,--,0,218.3,2.173,19.8406,0.0108,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--
270.6730185536942,-24.3365905701811,270.67301855369,17.5367,-24.33659057018,14.8840,4066020185958619904,--,--,--,--,--,--,0,90.79,2.416,20.7933,0.0289,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--


In [32]:
columns = ['_RAJ2000', '_DEJ2000', 'Source']
newtbl = ds9tbl[columns]
newtbl['ObjType'] = 'star'
newtbl.write(r"C:\Users\Kevin\repos\ReipurthBallyProject\data\M8\N-A-L656\region\newtbl.xml",format='votable')

OSError: File exists: C:\Users\Kevin\repos\ReipurthBallyProject\data\M8\N-A-L656\region\newtbl.xml

In [33]:
object_df.columns

Index(['thresh', 'npix', 'tnpix', 'xmin', 'xmax', 'ymin', 'ymax', 'x', 'y',
       'x2', 'y2', 'xy', 'errx2', 'erry2', 'errxy', 'a', 'b', 'theta', 'cxx',
       'cyy', 'cxy', 'cflux', 'flux', 'cpeak', 'peak', 'xcpeak', 'ycpeak',
       'xpeak', 'ypeak', 'flag'],
      dtype='object')

In [34]:
object_tbl = Table.from_pandas(object_df[['x','y','flux', 'npix', 'a', 'b', 'theta']])
object_tbl['x'] += 1
object_tbl['y'] += 1
object_tbl['eccentricity'] = object_df.b/object_df.a

In [35]:
object_tbl['ObjName'] = [f'Obj_{i:03d}' for i in range(len(object_df))]
object_tbl['ObjType'] = 'star'
object_tbl.write(r"C:\Users\Kevin\repos\ReipurthBallyProject\data\M8\N-A-L656\region\object_tbl.xml",
                 overwrite=True, format='votable')

In [36]:
deg = 0.1304870229507045
print(f'Minutes: {60.0*deg:.3f}')

Minutes: 7.829


In [38]:
def find_closest_ref( refs, point):
    pt = point.reshape(-1,2)
    offs = refs - pt
    dist = (offs **2).sum(axis=1)
    indx = dist.argmin()
    return {'index': indx, 'distance': np.sqrt(dist[indx]), 'x': refs[indx][0], 'y':refs[indx][1]}

In [39]:
find_closest_ref(dst, src[55])

{'index': 55,
 'distance': 28.69267941647783,
 'x': 443.63312191238106,
 'y': 1156.3857510315772}

In [40]:
dst[5]

array([141.04706814,  10.41951623])

In [41]:
fits.getval(img_path, 'MJD')

57898.595399

In [43]:
xx = np.arange(20).reshape(-1,2)
xx.sum(axis=1)

array([ 1,  5,  9, 13, 17, 21, 25, 29, 33, 37])

In [44]:
def printit(x,y):
    print(f'X: {x}, Y:{y}')

In [47]:
xx[3]=[44,55]
xx

array([[ 0,  1],
       [ 2,  3],
       [ 4,  5],
       [44, 55],
       [ 8,  9],
       [10, 11],
       [12, 13],
       [14, 15],
       [16, 17],
       [18, 19]])

In [45]:
for x in xx:
    printit(*x)

X: 0, Y:1
X: 2, Y:3
X: 4, Y:5
X: 6, Y:7
X: 8, Y:9
X: 10, Y:11
X: 12, Y:13
X: 14, Y:15
X: 16, Y:17
X: 18, Y:19


In [ ]:
def RMSE(xy_hat, xy_act):
    err = (xy_act - xy_hat)**2
    resid2 = err.sum(axis=1)
    rmse = np.sqrt(resid2.mean())
    return rmse

def initial_transorm(dirs, detector):

    coo_path = os.path.join(dirs['coord_maps'], detector+'.coo')

    coo_df = coo2df(coo_path)
    src = np.array([coo_df.x_in, coo_df.y_in]).T
    dst = np.array([coo_df.x_ref, coo_df.y_ref]).T

    tran = sk.transform.estimate_transform('polynomial', src, dst, 3)

    return tran

def align_image(dirs, inpath, outpath):

    maxiter = 10

    # get the image
    with fits.open(inpath) as f:
        img_hdr = f[0].header.copy()
        img_data = f[0].data.copy()
    img_data = img_data.astype(float)

    #get the intial transform and the objects in the image
    tran = initial_transorm(dirs, img_hdr['DETECTOR'])
    obj_df = find_objects(img_data, thresh=50.0)
    obj_xy = np.array([object_df.x, object_df.y]).T
    gaia_all = get_gaia(dirs, inpath)
    gaia_xy = np.zeros_like(obj_xy, dtype = np.float)

    print(f'Objects found: {len(obj_df)}')

    for iter in range(maxiter):
        obj_hat = tran(obj_xy)
        gaia_xy = np.array([find_closest_ref(gaia_all, obj) for obj in obj_xy])
        rmse[i] = RMSE(obj_hat, gaia_xy)


    img_new = sk.transform.warp(img_data, tran, cval = np.nan)
    img_new = img_new.astype(np.float32)

    #write out the result
    phdu = fits.PrimaryHDU(data = img_new, header=img_hdr)

    phdu.writeto(outpath, overwrite=True)
